In [1]:
with open("../data/day13.txt") as f:
    data = f.read().split("\n\n")

In [3]:
import re
import numpy as np
import tqdm

pattern = r"Button A: X\+(\d+), Y\+(\d+)\nButton B: X\+(\d+), Y\+(\d+)\nPrize: X=(\d+), Y=(\d+)"

parsed_data = []
for entry in data:
    match = re.match(pattern, entry)
    if match:
        button_a_x, button_a_y, button_b_x, button_b_y, prize_x, prize_y = map(
            int, match.groups()
        )
        parsed_data.append(
            {
                "Button A": np.array((button_a_x, button_a_y)),
                "Button B": np.array((button_b_x, button_b_y)),
                "Prize": np.array((prize_x, prize_y)),
            }
        )
    else:
        raise ValueError("")

In [ ]:
def calculate_minimal_tokens(machine):
    options = []
    for n_buttons in range(1, 201):
        tokens_a_min = max(n_buttons - 100, 0)
        tokens_a_max = min(n_buttons, 100)
        for buttons_a in range(tokens_a_min, tokens_a_max + 1):
            buttons_b = n_buttons - buttons_a
            total = buttons_a * machine["Button A"] + buttons_b * machine["Button B"]
            if (total == machine["Prize"]).all():
                options.append((buttons_a, buttons_b))
    if options:
        if len(options) > 2:
            print(options)
        costs = [buttons_a * 3 + buttons_b for buttons_a, buttons_b in options]
        return min(costs)

    return 0


total_costs = [calculate_minimal_tokens(machine) for machine in tqdm.tqdm(parsed_data)]

In [ ]:
sum(total_costs)

# Part 2

In [5]:
# EXAMPLE
# n_buttons_a * 69 + n_buttons_b * 27 = value_x
# n_buttons_a * 23 + n_buttons_b * 71 = value_y

# nbuttons_a * 69 * 71 + n_buttons * 27 * 71 = 71 * value_x
# ==> n_buttons_a * 69 * 71 + 27 * (value_y - n_buttons_a * 23) = 71 * value_x
# ==> n_buttons_a * (69 * 71 * value_x - 27 * value_y71 - 23 * 27) = 71 * value_x - 27 * value_y

# n_buttons_a =  () / (69 * 71 - 23 * 27)
# n_buttons_b = (value_x - 69 * n_buttons_a) / 27

# GENERAL
# n_buttons_a * a_x + n_buttons_b * b_x = value_x
# n_buttons_a * a_y + n_buttons_b * b_y = value_y
#
# n_buttons_a = b_y * value_x - b_x * value_y / (a_x * b_y - a_y * b_x)
# n_buttons_b = (value_x  - n_buttons_a * a_x) / b_x (or equivalently for value_y)

In [6]:
def calculate_tokens_large(machine):
    value_x, value_y = machine["Prize"] + 10000000000000
    a_x, a_y = machine["Button A"]
    b_x, b_y = machine["Button B"]
    n_buttons_a = (b_y * value_x - b_x * value_y) / (a_x * b_y - a_y * b_x)
    n_buttons_b = (value_x - n_buttons_a * a_x) / b_x
    if not n_buttons_a.is_integer() or not n_buttons_b.is_integer():
        return 0
    return n_buttons_a * 3 + n_buttons_b

In [7]:
total_costs_large = [calculate_tokens_large(machine) for machine in (parsed_data)]

In [ ]:
sum(total_costs_large)